In [13]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import Dataset
from datasets import load_dataset

load_dotenv(override=True)

True

## Get documents from HF

In [2]:
# Load from hub
ds_vejledninger = load_dataset(
    "jealk/dk_retrieval_benchmark",
    "retsinformation",
    split="train",
    #download_mode="force_redownload",
)

In [3]:
# Create pandas dataframe from the dataset using the huggingface datasets library
df_vejledninger = ds_vejledninger.to_pandas()
df_vejledninger.head()

,url,title,html_content,text_content
0,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om regulering af satser fra 1. janu...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om regulering af satser fra 1. janu...
1,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om satser i 2024 for betaling af ud...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om satser i 2024 for betaling af ud...
2,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om obligatorisk selvbooking af jobs...,"<div class=""document-content"" id=""restylingRoo...",Vejledning om obligatorisk selvbooking af jobs...
3,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning til bekendtgørelse om tilskud til s...,"<div class=""document-content"" id=""restylingRoo...",Vejledning til bekendtgørelse om tilskud til s...
4,https://www.retsinformation.dk/eli/retsinfo/20...,Vejledning om fleksløntilskud m.v.,"<div class=""document-content"" id=""restylingRoo...",Vejledning om fleksløntilskud m.v.\n1.Indledni...


## Function overview

- Step 0: Chunking text
    - Include or not?
- Step 1: Filter chunks
    - W. Textdescriptives
    - W. LLM call, egnet til spørgsmål?
- Step 2: Generate questions:
    - Using LLamaIndex
- Step 3: Filter generated questions
    - (Text descriptives for long texts)
    - LLM call: Is the answer found in chunk?
    - LLM call: Is the answer clear and in a natural language?
- Step 4: Update chunk-question table
    - Embed chunks, embed questions (Local Vector DB)
    - Use vector search to identify top 10 matches
    - (Optional, Rerank)
    - Filtering: Flag query/chunks where intended match is not in Top @10
    - If question/chunk not @1
        - Use LLM to check any question/chunk scored > than "real" match
        - Update Match Matrix if OK
    - If Delta simililarity score from 'real match' to other top @10 is < threshold:
        - Use LLM to check question/chunk
        - Update Match Matrix if OK
- Step 5: Convert to BEIR format

# Step 0

In [4]:
from typing import List, Dict, Any
from llama_index.core import Document


def create_documents(text: List[str], metadata: List[Dict[str, Any]]) -> List[Document]:
    """Create a list of llama_index documents from a list of strings and a list of dictionaries

    Args:
    text: A list of strings containing the text of the documents, eg. ["Vejledning om ...", "..."]
    metadata: A list of dictionaries containing one or multiple metadata, eg. [{"title": "Example 1", "source": "website_url"}, {...}]

    Returns:
    A list of llama_index documents
    """
    documents = [
        Document(text=content, metadata=meta) for content, meta in zip(text, metadata)
    ]
    return documents

In [5]:
llama_documents = create_documents(df_vejledninger["text_content"], df_vejledninger[["title", "url"]].to_dict(orient="records"))

In [6]:
from llama_index.core.node_parser import SentenceSplitter
from transformers import AutoTokenizer
from llama_index.core.schema import TextNode


def document_splitter(
    documents: List[Document],
    chunk_size: int = 512,
    tokenizer=AutoTokenizer.from_pretrained("intfloat/e5-base-v2"),
) -> List[TextNode]:
    """Split a list of llama_index documents into nodes

    Args:
    documents: A list of llama_index documents
    chunk_size: An integer defining the maximum number of tokens in each node
    tokenizer: A tokenizer from the Hugging Face transformers library

    Returns:
    A list of nodes, consisting of text, metadata, embeddings and node-relations
    """
    node_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=0,
        secondary_chunking_regex=str(["\n"]),
        paragraph_separator=str(["\n\n"]),
        tokenizer=tokenizer.tokenize,
    )
    nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)
    return nodes


nodes_vejledninger = document_splitter(llama_documents)

Parsing nodes: 100%|██████████| 433/433 [01:10<00:00,  6.15it/s]


# Step 1

**Filtering using text descriptives**

In [7]:
nodes_vejledninger_sample = nodes_vejledninger[:100]

In [8]:
import textdescriptives as td
import spacy

def filter_nodes_by_td(nodes: List[TextNode], filter_type: bool=True) -> List[TextNode]:
    """Filter nodes by the textdescriptives quality check

    Args:
    nodes: A list of llama_index nodes
    fiter_type: A boolean defining whether to filter by nodes that passed (True) or failed (False) the textdescriptives quality check

    Returns:
    A list of llama_index nodes that passed the textdescriptives quality check
    """
    nlp = spacy.blank("da")
    nlp.add_pipe("sentencizer")
    quality_pipe = nlp.add_pipe("textdescriptives/quality")
    docs = list(nlp.pipe([node.text for node in nodes]))
    filtered_nodes = [node for node, doc in zip(nodes, docs) if doc._.passed_quality_check==filter_type]
    
    return filtered_nodes


In [9]:
#filter
nodes_passed_td = filter_nodes_by_td(nodes_vejledninger_sample)

**Filtering using LLM call**

In [10]:
#Gample prompts

#system prompt
"""Din opgave er at evaluere hvorvidt et uddrag af en tekst, er egnet til at stille et generelt spørgsmål til.
    Du skal vurdere om uddraget indeholder klare og faktuelle informationer, hvorfra der kan formuleres et præcist, naturligt og kort spørgsmål der kan besvares ud fra uddraget.
    Du skal give scoren 1 til teksten, hvis der kan opstilles et naturligt formuleret spørgsmål til uddraget, som eksempelvis kunne bruges i sammenhæng med en eksamen eller test.
    Du skal give scoren 0 til teksten, hvis uddraget ikke indeholder generel faktuel information, hvis teksten er for usammenhængende eller detaljeret til at kunne formulere et generelt spørgsmål.
    Returner en json med key: llm_score og value i form af en int: "0" eller "1".
"""

#Few shot user prompt
""" Du er en erfaren sagsbehandler, nedenfor er eksempler på tekstuddrag og deres tilhørende score.

    ============================
    Start på eksempel, som skal have scoren 1:

    "Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde Indledning Efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr. 1541 af 12. december 2023, og lov om sikring mod følger af arbejdsskade, jf. lovbekendtgørelse nr. 943 af 16. oktober 2000, skal der med virkning fra 1. januar 2024 efter indstilling fra bestyrelsen for Arbejdsmarkedets Erhvervssikring ske regulering af lovens årslønsbeløb, godtgørelsesbeløb, overgangsbeløb samt løbende erstatninger. Reguleringen af satserne fastsættes af Arbejdstilsynets direktør efter bemyndigelse fra beskæftigelsesministeren. Satser efter loven reguleres med 2 procent tillagt tilpasningsprocenten for finansåret 2024 (jf. lov om en satsreguleringsprocent)."

    Fordi uddraget indeholder klare og faktuelle informationer, hvorfra der kan formuleres et præcist, naturligt og kort spørgsmål der kan besvares ud fra uddraget, eksempelvis:

    "Hvem fastsætter reguleringen af satserne for arbejdsskadesikring og andre relaterede ydelser?"

    ============================
    Start på eksempel, som skal have scoren 0:

    "årligt. 8)Uddannelsesgodtgørelsen i en forlænget periode efter § 18 b, stk. 3, 2. pkt., udgør 244.140 kr. årligt. Grundlønnen for beregning og regulering af løbende erstatning og uddannelsesgodtgørelse for arbejdsskader indtruffet den 1. juli 2024 eller senere er den efter lov om arbejdsskadesikring § 24 fastsatte årsløn multipliceret med 608.000/608.000, jf. § 24 a. Fastsættes en løbende erstatning eller en uddannelsesgodtgørelse den 1. juli 2024 eller senere, udbetales erstatningen eller godtgørelsen fra tidspunktet for dennes begyndelse med et tillæg på 0,0 pct. til den erstatning eller godtgørelse, der svarer til grundlønnen. Satser for arbejdsskader indtruffet i tiden 1. januar 2024 til 30. juni 2024 Med virkning for arbejdsskader efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr."

    Fordi der uddraget ikke indeholder meget detaljerede informationer der er svære at forstå uden kontekst, generelt er fragmenteret og gør det vanskeligt at formulere et generelt spørgsmål.

    ==============================
    Uddrag som du skal give en score:

    {chunk_text}"

    Returner KUN tallet 0 eller 1, ingen yderligere forklaring
"""
    

' Du er en erfaren sagsbehandler, nedenfor er eksempler på tekstuddrag og deres tilhørende score.\n\n    ============================\n    Start på eksempel, som skal have scoren 1:\n\n    "Vejledning om regulering af satser fra 1. januar 2024 efter lov om arbejdsskadesikring, lov om sikring mod følger af arbejdsskade, lov om arbejdsskadeforsikring og lov om forsikring mod følger af ulykkestilfælde Indledning Efter lov om arbejdsskadesikring, jf. lovbekendtgørelse nr. 1186 af 19. august 2022 med de ændringer, der følger af lov nr. 1541 af 12. december 2023, og lov om sikring mod følger af arbejdsskade, jf. lovbekendtgørelse nr. 943 af 16. oktober 2000, skal der med virkning fra 1. januar 2024 efter indstilling fra bestyrelsen for Arbejdsmarkedets Erhvervssikring ske regulering af lovens årslønsbeløb, godtgørelsesbeløb, overgangsbeløb samt løbende erstatninger. Reguleringen af satserne fastsættes af Arbejdstilsynets direktør efter bemyndigelse fra beskæftigelsesministeren. Satser efter 

In [47]:
import json
import logging
from typing import Dict, Any

from openai import OpenAI
client = OpenAI()

def q_eval_system_prompt():
    sys_prompt = """Din opgave er at evaluere et givet tekstuddrag for at bestemme, om det er egnet til at danne grundlag for et generelt spørgsmål, der er relevant for eksempelvis en eksamen eller en test. 
    For at vurdere dette, skal du fokusere på følgende tre nøglekriterier:

    1. Klarhed: Vurder, om teksten er formuleret klart og direkte, således at et spørgsmål til denne tekst, vil kunne besvares uden yderligere forklaringer. Teksten skal være læsbar og ikke usammenhængende i sin struktur.
    
    2. Konkret Information: Afgør, om uddraget indeholder specifikke, faktuelle informationer, der kan danne grundlag for et præcist og direkte spørgsmål. Teksten skal præsentere håndgribelige fakta eller data, som et spørgsmål kan baseres på.

    3. Kontekstuel Helhed: Bedøm, om teksten leverer tilstrækkelig kontekst for at et spørgsmål baseret på uddraget vil være meningsfuldt og forståeligt uden behov for yderligere information. Teksten skal være selvstændig og give en fuld forståelse af det emne, der behandles.

    Baseret på din evaluering:

    - Tildel scoren 1, hvis tekstuddraget opfylder alle tre kriterier, og der kan formuleres et naturligt, klart og kontekstuelt meningsfuldt spørgsmål baseret på teksten.

    - Tildel scoren 0, hvis tekstuddraget ikke opfylder et eller flere af de ovenstående kriterier, hvilket gør det uegnet til at danne grundlag for et generelt spørgsmål.
    """
    return sys_prompt

def q_eval_user_prompt(text: str) -> str:
    """Prepare the prompt for the API call."""
    
    qa_egnet_tmlp = """Du er en erfaren sagsbehandler. 
    Din Opgave:
    Vurder det følgende tekstuddrag og angiv, om det er egnet til at stille et generelt spørgsmål til.

    Uddrag:
    {chunk_text}
    
    Returner din vurdering i følgende JSON-format:

    {{
    "llm_score": [indsæt enten 0 eller 1 her]
    }}
    """
    return qa_egnet_tmlp.format(chunk_text=text)


def json_api_call(system_prompt: str, user_prompt: str, oai_model: str="gpt-3.5-turbo-0125") -> Dict[str, Any]:
    """Perform the API call to evaluate the text."""
    try:
        completion = client.chat.completions.create(
            model=oai_model,
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user", 
                    "content": user_prompt
                },
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)
    except json.JSONDecodeError as e:
        logging.error(f'JSON parsing failed: {e}')
    except Exception as e:
        logging.error(f'API call failed: {e}')
    return {}

def filter_nodes_by_llm(node_list: List[TextNode]) -> List[TextNode]:
    """Filter nodes by the llama quality check

    Args:
    nodes: A list of llama_index nodes
    
    Returns:
    A list of llama_index nodes that passed the llama quality check
    """
    nodes_passed_list = []
    system_prompt = q_eval_system_prompt()
    for llama_node in node_list:
        user_prompt = q_eval_user_prompt(llama_node.text)
        response = json_api_call(system_prompt, user_prompt)
        if response:
            if response['llm_score'] == 1:
                nodes_passed_list.append(llama_node)
            else:
                continue
        else:
            logging.error(f'Failed to evaluate below node due to an earlier error. \n {llama_node.text}')
    return nodes_passed_list

In [23]:
nodes_passed_llm = filter_nodes_by_llm(nodes_passed_td[:20])

In [25]:
len(nodes_passed_llm)

16

In [36]:
#Run on the last 20 nodes in the list
nodes_passed_llm_last = filter_nodes_by_llm(nodes_passed_td[-20:])

## Experiment to compare how LLM filtering performs compared to text descriptives

In [ ]:
#Create a list of nodes that fail the textdescriptives quality check
nodes_failed_td = filter_nodes_by_td(nodes_vejledninger_sample, filter_type=False)
len(nodes_failed_td)

In [ ]:
print(f'Number of nodes that passed the textdescriptives quality check: {len(nodes_passed_td)}')
print(f'Number of nodes that failed the textdescriptives quality check: {len(nodes_failed_td)}')

In [ ]:
#Sample first 100 each node type
nodes_failed_td_sample = nodes_failed_td[:20]
nodes_passed_td_sample = nodes_passed_td[:20]

#Evaluate the nodes
evaluate_node_list(nodes_failed_td_sample)
evaluate_node_list(nodes_passed_td_sample)

In [ ]:
#Count the number of llm_scores for each group using Counter
from collections import Counter
import numpy as np

passed_td_meta = [node.metadata for node in nodes_passed_td_sample]
failed_td_meta = [node.metadata for node in nodes_failed_td_sample]

passed_td_llm_count = dict(Counter([meta["llm_score"] for meta in passed_td_meta]))
failed_td_llm_count = dict(Counter([meta["llm_score"] for meta in failed_td_meta]))

#Plot a confusion matrix where x-axis is the Truth, LLM evaluation, and y-axis is the Predicted, Textdescriptives quality check
#The matrix is a 2x2 matrix with the following values:
TP = passed_td_llm_count[1] if 1 in passed_td_llm_count else 0
FP = passed_td_llm_count[0] if 0 in passed_td_llm_count else 0
TN = failed_td_llm_count[0] if 0 in failed_td_llm_count else 0
FN = failed_td_llm_count[1] if 1 in failed_td_llm_count else 0

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

confusion_matrix = np.array([[TP, FP], [FN, TN]])

sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Passed", "Failed"],
            yticklabels=["Passed", "Failed"])
plt.xlabel('LLM eval')
plt.ylabel('Textdescriptives')
plt.title('Confusion Matrix')
plt.show()

**Conclusion after inspecting 40 samples**
General alignment between LLM and text descriptives. 
- 80% of those texts that pass TD also pass LLM (true positives)
- 60% of those who fail TD also fail LLM (true negatives)
- A fairly high rate of false negatives in the sense that 40% of those who are filtered out by TD, are according to LLM eval suitable for questions. 

The cause of those false negatives seem mostly to stem from text containing many formulas or lists, ex. below:

*Månedlig erstatning: 8.625 kr.\nc.Uddannelsesgodtgørelse\nSkadedato: 2. juli 2024. Årsløn: 425.000 kr.\nGrundløn: 425.000 kr. × 608.000/608.000 = 425.000 kr.\nGrundydelse: 0,83 × 425.000 kr. = 352.750,00 kr. årligt.\nÅrlig godtgørelse fra 1. juli 2024: 352.750,00 kr. × 1,000 = 352.750,00 kr., der forhøjes til nærmeste med 12 delelige kronebeløb: 352.752 kr.\nMånedlig godtgørelse: 29.396 kr.\n1.2 Engangsbeløb\nSkadedato: 8. juli 2024. Afgørelsesdato: 6. december 2024. Årsløn: 350.000 kr. Tab af erhvervsevne: 25 pct. Varigt mén: 20 pct. Alder ved afgørelsen: 35 år og 2 måneder.\na.Erstatning for tab af erhvervsevne\nGrundløn: 350.000 kr. × 608.000/608.000 = 350.000 kr.\nGrundydelse: 0,25 × 0,83 × 350.000 kr. × 0,92 = 66.815,00 kr. årligt.\nÅrlig erstatning fra 1. januar 2024: 66.815,00 kr.*

# Step 2
### Generate questions

In [41]:
from llama_index.core.prompts import PromptTemplate

# Define your custom prompt template in Danish
qa_sagsbehandler_prompt = """Nedenfor er et uddrag (kontekst) fra en længere tekst:
---------------------
{context_str}
---------------------
Givet ovenstående uddrag og ingen forudgående viden, er din opgave at generere præcis {num_questions_per_chunk} spørgsmål til teksten.
En sætning skal kun indeholde 1 spørgsmål, og spørgsmålet skal være formuleret kort og præcist. 
Svaret til spørgsmålet, skal kunne findes i ovenstående uddrag.
Spørgsmålet skal indeholde specifik kontekst, således at spørgsmålet efterfølgende kan besvares entydigt og uden kendskab til uddraget. 
Spørgsmålene skal stilles i et sprog som en borger uden juridisk ekspertise kan forstå.

Eksempel på et spørgsmål der ikke har en specifik kontekst, og som fejlagtigt indeholder 2 spørgsmål i 1 sætning: 
"Hvilket dokument har den nye vejledning erstattet, og hvornår blev den udsendt?" -Da det ikke angivet hvilket dokument der er tale om, og derfor er svaret til spørgsmålet ikke entyidgt, uden kendskab til uddraget. Sætningen indeholder desuden 2 spørgsmål i samme sætning. 

Eksempel på et godt spørgsmål, som kan besvares entydigt uden kendskab til uddraget:
"Hvilke to indbetalinger udgør det samlede medlemsbidrag til en a-kasse?" - Da det er klart hvad der spørges om, og der kun er 1 rigtigt svar i den givne lovtekst.
"""

def llama_prompt_template(prompt_template: str) -> PromptTemplate:
    return PromptTemplate(prompt_template)
    
    
qa_sagsbehandler_tmlp = llama_prompt_template(qa_sagsbehandler_prompt)

In [42]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.llms.openai import OpenAI

#wrap function to generate qa pairs
def generate_questions(nodes: List[TextNode], qa_generate_prompt_tmpl: PromptTemplate, num_questions_per_chunk: int=1, llm = OpenAI(temperature=0.0, model="gpt-4-0125-preview")) -> Dataset:
    qa_dataset = generate_qa_embedding_pairs(
        qa_generate_prompt_tmpl=qa_generate_prompt_tmpl,
        llm=llm,
        nodes=nodes,
        num_questions_per_chunk=num_questions_per_chunk,
    )
    return qa_dataset


In [29]:
qa_dataset = generate_questions(nodes_passed_llm, qa_sagsbehandler_tmlp)
qa_dataset.queries

{'6ad1366c-ad20-4815-aea8-f3f03cb3f98e': '"Hvem fastsætter reguleringen af satserne for arbejdsskadesikring fra 1. januar 2024, og hvem giver bemyndigelsen til dette?"',
 'eb69123b-cb35-4535-80bb-5b9714976fab': '"Hvordan beregnes grundlønnen for løbende erstatninger for tab af erhvervsevne og tab af forsørger ifølge Arbejdstilsynets vejledning fra den 5. januar 2024?"',
 '28bcc695-2917-44bb-b32b-a50299d8aa3c': 'Hvordan beregnes grundlønnen for løbende erstatninger for tab af erhvervsevne og tab af forsørger samt uddannelsesgodtgørelser for arbejdsskader indtruffet mellem 1. januar 2004 og 31. december 2010?',
 'c5156a1b-4bfe-4862-b7b3-e7133caab286': 'Hvordan beregnes kapitalerstatningen for tab af erhvervsevne ifølge den lov, der blev affattet den 12. december 2023?',
 '9b4de951-63df-45b0-8ceb-c77037311ec1': '"Hvad er den nye fastsatte procentdel for grundydelsen pr. 1. januar 2024 i forhold til den løbende erstatning, der svarer til grundlønnen fra 1. april 1994?"',
 '8604d870-437d-47

In [44]:
qa_dataset = generate_questions(nodes_passed_llm, qa_sagsbehandler_tmlp)
qa_dataset.queries

100%|██████████| 16/16 [00:34<00:00,  2.14s/it]


{'3a6fd8e4-e18f-45b6-80e3-4c2bb9a64ed3': 'Hvem fastsætter reguleringen af satserne efter loven om arbejdsskadesikring med virkning fra 1. januar 2024?',
 'd8ad4a00-2b68-4b28-93cd-af909189a262': 'Hvordan beregnes grundlønnen for løbende erstatninger ifølge Arbejdstilsynets vejledning fra den 5. januar 2024?',
 'fa4fa371-e72e-40e1-8075-bb230f4ba0ea': 'Hvordan beregnes grundlønnen for løbende erstatninger for tab af erhvervsevne og tab af forsørger ifølge lovbekendtgørelsen fra 7. september 2009?',
 '3dae4fc0-3ff5-48c6-8ad9-7a4698cb35d9': 'Hvad er den forhøjede værdi af méngodtgørelsen for et varigt mén på 100 procent, som nævnt i loven fra 12. december 2023?',
 '505d1fcc-ca6d-4d7c-8453-0d967e66c8c3': 'Hvad er den nye procentdel for grundydelsen pr. 1. januar 2024 i forhold til den løbende erstatning, der svarer til grundlønnen fra 1. april 1994?',
 '0dede3cb-15a3-4e7c-b387-9455059aecbf': 'Hvilke målgrupper er omfattet af pligten til selvbooking af jobsamtaler ifølge vejledningen fra maj 

In [43]:
qa_dataset_last = generate_questions(nodes_passed_llm_last, qa_sagsbehandler_tmlp)
qa_dataset_last.queries

100%|██████████| 17/17 [00:35<00:00,  2.12s/it]


{'d9acf241-fcd5-4dcf-b257-4b32a4a4fdf4': 'Hvilke målgrupper er omfattet af pligten til selvbooking ifølge vejledningen fra lovbekendtgørelse nr. 701 af 22. maj 2022?',
 'e3dcc8d8-4830-4007-932b-118f7bd23bec': 'Hvem har ansvaret for kontaktforløbet for dagpengemodtagere i målgruppen for uddannelsespålæg?',
 'cb56ecec-b8f4-41dc-9aab-60e084261c6c': 'Hvor mange individuelle jobsamtaler med jobcenteret skal en person i ledighed have inden for de første 6 måneder?',
 '7b098644-0aec-4a8b-b64f-ce19b038c1ad': 'Hvem har ansvaret for at aftale det videre kontaktforløb efter de første 6 måneders ledighed?',
 'bff5e4fe-1eae-48a2-9a68-7ff58a5d9906': 'Hvad kan en person ikke vælge som form for samtale ifølge § 33, stk. 1, 2. pkt.?',
 'b79a02e7-f964-43b1-be8b-66472963c8fa': 'Hvornår kan jobcenteret beslutte at ændre en personens ønskede samtaleform fra telefonisk eller video til personligt fremmøde?',
 'a0b148d2-3dac-45a7-b82b-190ed9961797': 'Hvordan kan jobsamtaler for personer på barsel afholdes, hv

### Alternative instead of using LLamaIndex (WIP)

In [17]:
def generate_question_template(text: str, num_q: int=1) -> str:
    question_tmlp = """Nedenfor er et uddrag fra en længere tekst:
    ---------------------
    {context_str}
    ---------------------
    Givet ovenstående uddrag og ingen forudgående viden, er din opgave at generere spørgsmål til teksten.
    Svaret til spørgsmålet, skal kunne findes i ovenstående uddrag.
    Spørgsmålet skal indeholde specifik kontekst, således at spørgsmålet kan besvares entydigt og uden kendskab til uddraget. 
    
    Du er en erfaren sagsbehandler, og din opgave er at stille præcis {num_questions_per_chunk} spørgsmål.
    Spørgsmålene skal stilles i et sprog som en borger uden juridisk ekspertise kan forstå.
    
    Eksempel på et spørgsmål der ikke har en specifik kontekst: 
    "Hvilket dokument har den nye vejledning erstattet, og hvornår blev den udsendt?" -Da det ikke angivet hvilket dokument der er tale om, og derfor er svaret til spørgsmålet ikke entyidgt, uden kendskab til uddraget. 
    
    Eksempel på et godt spørgsmål, som kan besvares entydigt uden kendskab til uddraget:
    "Hvilke to indbetalinger udgør det samlede medlemsbidrag til en a-kasse?" - Da det er klart hvad der spørges om, og der kun er 1 rigtigt svar i den givne lovtekst.
    
    Returner i json format med key: q og value: en liste af gode spørgsmål:

    {{
    "q": ["spørgsmål 1", "spørgsmål 2", "..."]
    }}
    """
    return question_tmlp.format(context_str=text, num_questions_per_chunk=num_q)

In [18]:
def question_api_call(user_prompt: str, oai_model: str="gpt-4-0125-preview") -> Dict[str, Any]:
    """Perform the API call to evaluate the text."""
    try:
        completion = client.chat.completions.create(
            model=oai_model,
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": "Din opgave er at stille præcise spørgsmål til et givet tekstuddrag og returnere en json med en liste af spørgsmål."
                },
                {
                    "role": "user", 
                    "content": user_prompt
                },
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(completion.choices[0].message.content)
    except json.JSONDecodeError as e:
        logging.error(f'JSON parsing failed: {e}')
    except Exception as e:
        logging.error(f'API call failed: {e}')
    return {}

In [19]:
def generate_questions(text: str, num_q: int=1) -> Dict[str, Any]:
    """Generate questions for a chunk of text."""
    user_prompt = generate_question_template(text, num_q=num_q)
    response = question_api_call(user_prompt)
    return response

In [27]:
q1 = generate_questions(nodes_passed_llm[0].text, 2)
q1

{'q': ['Hvem fastsætter reguleringen af satserne for arbejdsskadesikring med virkning fra 1. januar 2024?',
  'Hvordan beregnes reguleringen af satserne for arbejdsskadesikring, der træder i kraft den 1. januar 2024?']}

## Step 3, Question filtering

In [31]:
from copy import deepcopy

def filter_qa_length(qa_dataset, char_max: int=150) -> Dataset:
    # Step 1: Create a deep copy of the entire dataset
    qa_copy = deepcopy(qa_dataset)
    
    # Step 2: Identify queries to remove based on length
    queries_to_remove = [query_id for query_id, query_text in qa_copy.queries.items() if len(query_text) > char_max]
    
    # Step 3: Identify documents to remove associated with the queries
    docs_to_remove = set()
    for query_id in queries_to_remove:
        associated_docs = qa_copy.relevant_docs.get(query_id, [])
        docs_to_remove.update(associated_docs)
    
    # Step 4: Remove the identified queries and documents
    for query_id in queries_to_remove:
        del qa_copy.queries[query_id]  # Remove query
        del qa_copy.relevant_docs[query_id]  # Remove entry from relevant_docs
    
    for doc_id in docs_to_remove:
        del qa_copy.corpus[doc_id]  # Remove associated document
    
    return qa_copy

In [59]:
filtered_qa_dataset = filter_qa_length(qa_dataset, char_max=150)
filtered_qa_dataset.queries

{'3a6fd8e4-e18f-45b6-80e3-4c2bb9a64ed3': 'Hvem fastsætter reguleringen af satserne efter loven om arbejdsskadesikring med virkning fra 1. januar 2024?',
 'd8ad4a00-2b68-4b28-93cd-af909189a262': 'Hvordan beregnes grundlønnen for løbende erstatninger ifølge Arbejdstilsynets vejledning fra den 5. januar 2024?',
 'fa4fa371-e72e-40e1-8075-bb230f4ba0ea': 'Hvordan beregnes grundlønnen for løbende erstatninger for tab af erhvervsevne og tab af forsørger ifølge lovbekendtgørelsen fra 7. september 2009?',
 '3dae4fc0-3ff5-48c6-8ad9-7a4698cb35d9': 'Hvad er den forhøjede værdi af méngodtgørelsen for et varigt mén på 100 procent, som nævnt i loven fra 12. december 2023?',
 '505d1fcc-ca6d-4d7c-8453-0d967e66c8c3': 'Hvad er den nye procentdel for grundydelsen pr. 1. januar 2024 i forhold til den løbende erstatning, der svarer til grundlønnen fra 1. april 1994?',
 '0dede3cb-15a3-4e7c-b387-9455059aecbf': 'Hvilke målgrupper er omfattet af pligten til selvbooking af jobsamtaler ifølge vejledningen fra maj 

In [62]:
filtered_qa_dataset.relevant_docs['0c16fd7a-dec8-4fb1-aac4-48a6fef195a8']

['60df62c2-b207-4ad5-a47c-ff87e94419c0']

In [63]:
filtered_qa_dataset.corpus['60df62c2-b207-4ad5-a47c-ff87e94419c0']

'Formål\nSelvbooking af jobsamtaler har til formål at give personen ansvaret for sit eget ledighedsforløb, og at personen har medindflydelse på, hvornår jobsamtalen skal foregå. Det medvirker til at styrke personens muligheder for at komme i job.\nSelvbooking skal ske digitalt. Selvbooking kan foregå via Jobnet, men det kan også være muligt via andre løsninger, fx via kommunens egen hjemmeside eller fremmødestandere. Der er således ikke krav om, at selvbooking foregår via Jobnet. Baggrunden herfor er, at en binding til Jobnet kan fastlåse en arbejdsdeling mellem digitale løsninger og aktørerne, der med tiden måtte vise sig ikke at være hensigtsmæssig. Derudover kan personen tilbydes en bedre service, når personen har mulighed for at booke møder ad flere kanaler end Jobnet.\nFor dagpengemodtagere vil selvbooking af jobsamtaler i arbejdsløshedskassen foregå gennem arbejdsløshedskassens digitale bookingsystem, dvs. typisk via arbejdsløshedskassens "Min Side".\n\n1.1.Jobsamtaler i kontaktf

In [81]:
from openai import OpenAI
client = OpenAI()

def q_simplify_system_prompt():
    sys_prompt = """Din opgave er simplificere et givet spørgsmål, så det er kortere, mere præcist og formuleret i et mere naturligt sprog. Fjern detaljer omkring specifikke paragraf, datoer osv."""
    return sys_prompt

def q_simplify_question_prompt(text: str) -> str:
    """Prepare the prompt for the API call."""
    
    qa_egnet_tmlp = """
    
    Eksempel på et eksisterende spørgsmål: 'Hvor mange individuelle jobsamtaler med jobcenteret skal en person i ledighed have inden for de første 6 måneders ledighed i henhold til loven fra 1. januar 2024?'
    Som kan omformuleres til: 'Hvor mange jobsamtaler skal en ledig have med jobcenteret inden for de første 6 måneder?'
    
    Nedenfor er et spørgsmål, som du skal analysere og hvis nødvendigt, omformulere:

    Spørgsmål:
    {chunk_text}
    
    Returner det omformulerede spørgsmål i følgende JSON-format:
    
        {{
        "simplified_q": "Dit omformulerede spørgsmål"
        }}
    """
    return qa_egnet_tmlp.format(chunk_text=text)

def rephrase_query(text: str) -> str:
    """Rephrase a given query."""
    user_prompt = q_simplify_question_prompt(text)
    response = question_api_call(user_prompt)
    return response

In [82]:
sample_q = list(filtered_qa_dataset.queries.values())[0]
sample_q

'Hvem fastsætter reguleringen af satserne efter loven om arbejdsskadesikring med virkning fra 1. januar 2024?'

In [83]:
simplified_q = json_api_call(q_simplify_system_prompt(), q_simplify_question_prompt(sample_q))
simplified_q

{'simplified_q': 'Hvem fastsætter satserne for arbejdsskadesikringen fra 1. januar 2024?'}

In [ ]:
def filter_nodes_by_llm(node_list: List[TextNode]) -> List[TextNode]:
    """Filter nodes by the llama quality check

    Args:
    nodes: A list of llama_index nodes
    
    Returns:
    A list of llama_index nodes that passed the llama quality check
    """
    nodes_passed_list = []
    system_prompt = q_eval_system_prompt()
    for llama_node in node_list:
        user_prompt = q_eval_user_prompt(llama_node.text)
        response = json_api_call(system_prompt, user_prompt)
        if response:
            if response['llm_score'] == 1:
                nodes_passed_list.append(llama_node)
            else:
                continue
        else:
            logging.error(f'Failed to evaluate below node due to an earlier error. \n {llama_node.text}')
    return nodes_passed_list

'Hvem fastsætter reguleringen af satserne efter loven om arbejdsskadesikring med virkning fra 1. januar 2024?'